In [1]:
import os
path = "C:/pytest/"
os.chdir(path)

In [2]:
def read_data(filename, encoding = 'cp949',start = 1):
    with open(filename, 'r', encoding = encoding) as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[start:]
    return data
def write_data(data, filename, encoding = 'cp949'):
    with open(filename, 'w', encoding = encoding) as f:
        f.write(data)

In [3]:
data = read_data('ratings_morphed.txt', encoding ='cp949')

In [4]:
import random
import math
import numpy as np
import tensorflow as tf

In [5]:
data_text = [line[1] for line in data]
data_senti = [line[2] for line in data]

In [6]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data_text, data_senti, stratify = data_senti)

# Tokenizing

In [7]:
# 토큰화
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import math

In [8]:
max_words = 10000 # 데이터셋에서 가장 빈도 높은 10,000 개의 단어만 사용
                  # n-1개의 단어를 가져오는 데 이는 padding을 위한 단어 하나를 남겨 둠
maxlen = 20 # 20개 이후의 단어는 버려서 각 문장의 길이를 고정
            # 20보다 길면 뒤를 자르고, 작으면 0으로 채움
embedding_dim = 200

In [9]:
tokenizer = Tokenizer(num_words=max_words) # 상위빈도 10,000개 단어를 추려내는 Tokenizer 객체 생성
tokenizer.fit_on_texts(train_X) # 전체 단어를 대상으로 인덱스 구축(빈도 기준)
word_index = tokenizer.word_index # 단어 인덱스 가져오기

### Data Sequencing

In [10]:
# 텍스틀 숫자로 변환
train_X_sequencing = tokenizer.texts_to_sequences(train_X)

# Data Padding

In [11]:
train_X_padding = pad_sequences(train_X_sequencing, maxlen= maxlen)

In [12]:
train_y = np.array(train_y).astype('float32')

# 모델 설계

In [13]:
from keras.models import Sequential
from keras import layers

In [45]:
model = Sequential()
model.add(layers.Embedding(input_dim = max_words, output_dim = embedding_dim, input_length = maxlen))

model.add(layers.Conv1D(64, kernel_size = 3,activation = 'relu'))
# model.add(layers.MaxPooling1D(pool_size = 2)) # maxpooling + lstm : 82.8 / maxpooling+flatten : 84.1
model.add(layers.Flatten())
model.add(layers.Dropout(0.4)) # conv + dropout + lstm + dropout : 84.5

# model.add(layers.LSTM(64)) # 83.6

# model.add(layers.Dropout(0.4))
# model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 20, 200)           2000000   
                                                                 
 conv1d_11 (Conv1D)          (None, 18, 64)            38464     
                                                                 
 flatten_3 (Flatten)         (None, 1152)              0         
                                                                 
 dropout_13 (Dropout)        (None, 1152)              0         
                                                                 
 dense_18 (Dense)            (None, 1)                 1153      
                                                                 
Total params: 2,039,617
Trainable params: 2,039,617
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.compile(loss= 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['acc'])
with tf.device('/CPU:0'):
    history = model.fit(train_X_padding, train_y, epochs=10, batch_size=32, validation_split=0.3)
history_dict = history.history

Epoch 1/10
3242/3242 [==============================] - 20s 6ms/step - loss: 0.4128 - acc: 0.8103 - val_loss: 0.3775 - val_acc: 0.8322
Epoch 2/10
3242/3242 [==============================] - 20s 6ms/step - loss: 0.3628 - acc: 0.8435 - val_loss: 0.3649 - val_acc: 0.8403
Epoch 3/10
3242/3242 [==============================] - 20s 6ms/step - loss: 0.3463 - acc: 0.8539 - val_loss: 0.4102 - val_acc: 0.8206
Epoch 4/10
3242/3242 [==============================] - 20s 6ms/step - loss: 0.3332 - acc: 0.8617 - val_loss: 0.3971 - val_acc: 0.8301
Epoch 5/10
3242/3242 [==============================] - 20s 6ms/step - loss: 0.3194 - acc: 0.8694 - val_loss: 0.3753 - val_acc: 0.8398
Epoch 6/10
3242/3242 [==============================] - 21s 6ms/step - loss: 0.3056 - acc: 0.8764 - val_loss: 0.3838 - val_acc: 0.8395
Epoch 7/10
3242/3242 [==============================] - 20s 6ms/step - loss: 0.2902 - acc: 0.8843 - val_loss: 0.3975 - val_acc: 0.8358
Epoch 8/10
3242/3242 [==============================] -

# 테스트 데이터 처리
+ 앞선 훈련 데이터 처리한 것과 동일한 과정을 거쳐야 함

In [17]:
# sequencing
test_X_sequencing = tokenizer.texts_to_sequences(test_X)

In [18]:
# Padding
test_X_padding = pad_sequences(test_X_sequencing, maxlen = maxlen)

In [19]:
test_y = np.asarray(test_y).astype('float32')

In [47]:
with tf.device('/CPU:0'):
    test_eval = model.evaluate(test_X_padding, test_y)
print(test_eval)

1544/1544 [==============================] - 4s 3ms/step - loss: 0.4502 - acc: 0.8146
[0.450157105922699, 0.8145980834960938]
